In [1]:
import pandas as pd
import joblib

In [3]:
csv_test = pd.read_csv('./data.csv')
x_test = csv_test.drop(['id', 'diagnosis', 'Unnamed: 32'], axis=1)


In [4]:
import json

In [10]:
data = {
    "signature_name": "serving_default",
    "instances": x_test.to_numpy().tolist()
}

In [11]:
with open('./test_json.json', 'w') as outfile:
    json.dump(data, outfile)

In [12]:
joblib.load('./cancer_detection_model_svm.pkl')

/usr/local/lib/python3.6/dist-packages/sklearn/base.py:315: UserWarning: Trying to unpickle estimator SVC from version 1.0.2 when using version 0.24.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


SVC()

In [16]:
import pickle
with open('./cancer_detection_model_svm.pkl', 'rb') as f:
    data = pickle.load(f)

In [17]:
data

array(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error',
       'fractal dimension error', 'worst radius', 'worst texture',
       'worst perimeter', 'worst area', 'worst smoothness',
       'worst compactness', 'worst concavity', 'worst concave points',
       'worst symmetry', 'worst fractal dimension'], dtype=object)

In [31]:
from kfserving import V1alpha2SKLearnSpec
import kfserving
import os

In [26]:
os.environ.update({
    'S3_ENDPOINT'          : 'minio-service.kubeflow:9000',
    'AWS_ACCESS_KEY_ID'    : 'minio',
    'AWS_SECRET_ACCESS_KEY': 'minio123',
    'S3_USE_HTTPS'         : '0',   # Whether or not to use HTTPS. Disable with 0.                        
    'S3_VERIFY_SSL'        : '0'    # If HTTPS is used, controls if SSL should be enabled. Disable with 0.
})  

In [28]:
sklearn_spec = V1alpha2SKLearnSpec(storage_uri="s3://model/cancer2")

In [35]:
KFServing = kfserving.KFServingClient()

In [43]:
isvc = KFServing.get(name='cifa10')

In [44]:
isvc_url = isvc['status']['address']['url']

In [45]:
isvc['status']

{'address': {'url': 'http://cifa10.myspace.svc.cluster.local/v1/models/cifa10:predict'},
 'canary': {},
 'conditions': [{'lastTransitionTime': '2022-04-26T05:39:23Z',
   'status': 'True',
   'type': 'DefaultPredictorReady'},
  {'lastTransitionTime': '2022-04-26T05:39:23Z',
   'status': 'True',
   'type': 'Ready'},
  {'lastTransitionTime': '2022-04-26T05:39:23Z',
   'status': 'True',
   'type': 'RoutesReady'}],
 'default': {'predictor': {'host': 'cifa10-predictor-default.myspace.example.com',
   'name': 'cifa10-predictor-default-pkc7z'}},
 'traffic': 100,
 'url': 'http://cifa10.myspace.example.com'}